In [2]:
import cv2
import numpy as np

# Function to be called when mouse is moved
def mouse_move(event, x, y, flags, param):
    if event == cv2.EVENT_MOUSEMOVE:
        # Get pixel value
        pixel_value = img_gray[y, x]
        # Map the pixel value to a temperature range (adjust according to your needs)
        temperature = map_temperature(pixel_value)
        print(f"Temperature at ({x}, {y}): {temperature}°C")

def map_temperature(pixel_value):
    # Example of mapping, adjust the formula according to your requirements
    # Assuming 0-255 pixel values map to a temperature range of 0-100°C
    return (pixel_value / 255.0) * 1500

# Load the image in grayscale
img_gray = cv2.imread(r"C:\Users\YASH\Downloads\laddle3.png", cv2.IMREAD_GRAYSCALE)

# Apply a colormap for visualization
img_colored = cv2.applyColorMap(img_gray, cv2.COLORMAP_JET)

# Create a window and set a mouse callback function
cv2.namedWindow("Image")
cv2.setMouseCallback("Image", mouse_move)

# Display the image
while True:
    cv2.imshow("Image", img_colored)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# When everything done, release the window
cv2.destroyAllWindows()

Temperature at (718, 0): 194.11764705882354°C
Temperature at (717, 1): 200.0°C
Temperature at (715, 2): 211.76470588235293°C
Temperature at (714, 4): 211.76470588235293°C
Temperature at (712, 5): 217.64705882352942°C
Temperature at (710, 8): 217.64705882352942°C
Temperature at (710, 10): 211.76470588235293°C
Temperature at (707, 12): 211.76470588235293°C
Temperature at (706, 15): 217.64705882352942°C
Temperature at (704, 21): 211.76470588235293°C
Temperature at (704, 21): 211.76470588235293°C
Temperature at (702, 29): 205.8823529411765°C
Temperature at (702, 33): 205.8823529411765°C
Temperature at (702, 37): 223.52941176470588°C
Temperature at (702, 41): 252.94117647058826°C
Temperature at (702, 45): 229.41176470588238°C
Temperature at (702, 47): 217.64705882352942°C
Temperature at (702, 51): 235.2941176470588°C
Temperature at (702, 55): 223.52941176470588°C
Temperature at (700, 63): 217.64705882352942°C
Temperature at (698, 76): 217.64705882352942°C
Temperature at (697, 84): 317.64705

In [29]:
from __future__ import print_function
import os
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from datetime import datetime,date
import numpy as np

In [69]:

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SPREADSHEET_ID = '1AQ6TgA_hm2bocR8eEwC9WLBDc7vScjg2ZhKAwAvIIBQ'
SAMPLE_RANGE_NAME = 'Sheet1'
creds = None
if os.path.exists("token.json"):
    creds = Credentials.from_authorized_user_file("token.json",SCOPES)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file("credentials.json",SCOPES)
        creds = flow.run_local_server(port=0)
    with open("token.json","w") as token:
        token.write(creds.to_json())
service = build("sheets","v4", credentials=creds)
sheet = service.spreadsheets().get(spreadsheetId=SPREADSHEET_ID).execute()
sheets = sheet['sheets']
today = date.today()
today = str(today)
# Print sheet names
for sheet in sheets:
    sheet_title = sheet['properties']['title']
    if sheet_title == today:
        break
else:
    
# Create a new sheet with today's date as the name
    request_body = {
        'requests': [
            {
                'addSheet': {
                    'properties': {
                        'title': today
                    }
                }
            }
        ]
    }
    service.spreadsheets().batchUpdate(spreadsheetId= SPREADSHEET_ID , body=request_body).execute()
# 

sheet = service.spreadsheets()
result = sheet.values().get(spreadsheetId=SPREADSHEET_ID,range=f'{today}!A:F').execute()
value = result.get('values',[])
print(value)
print(type(value))
value = np.array(value)
print("Numpy matrix form \n",value)


  


[['1', '1', '11:07', '1', '11:07', '11:15'], ['1', '1', '11:07', '2', '11:15', '03:40']]
<class 'list'>
Numpy matrix form 
 [['1' '1' '11:07' '1' '11:07' '11:15']
 ['1' '1' '11:07' '2' '11:15' '03:40']]


In [70]:
r,c = value.shape
print(r,c)
print(type(value[r-1,c-1]))


2 6
<class 'numpy.str_'>


In [72]:
room = str(1)
time_now2 = datetime.now()
exit = time_now2.strftime('%H:%M:%S')
entry = "11:07"
initial_time = "11:00"
initial_room = 1
laddle_number = 1
if value[r-1,3]==room:
    print("yes")
    new_value = [[exit]]
    request_body = { 
        'values':new_value
    }
    sheet1 = service.spreadsheets()
    sheet1.values().update(spreadsheetId=SPREADSHEET_ID,range=f'{today}!F{r}',valueInputOption='USER_ENTERED',body= request_body).execute()
    print(new_value)
else:
    print("no")
    new_value = [[laddle_number,initial_room,initial_time,room,entry,exit]]
    request_body = { 
        'values':new_value
    }
    sheet1 = service.spreadsheets()
    sheet1.values().append(spreadsheetId=SPREADSHEET_ID,range=f'{today}!A:A',valueInputOption='USER_ENTERED',body= request_body).execute()
    print(new_value)

no
[[1, 1, '11:00', '1', '11:07', '03:43:49']]


In [58]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Replace 'YOUR_SPREADSHEET_ID' and 'YOUR_SHEET_NAME' with your actual spreadsheet ID and sheet name
SPREADSHEET_ID = '1AQ6TgA_hm2bocR8eEwC9WLBDc7vScjg2ZhKAwAvIIBQ'
SHEET_NAME = 'vizag'

# Replace 'YOUR_JSON_KEY_FILE.json' with the path to your Google Sheets API service account key JSON file
JSON_KEY_FILE = 'token.json'

# Authenticate with Google Sheets API
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name(JSON_KEY_FILE, scope)
gc = gspread.authorize(credentials)

# Open the specified spreadsheet and sheet
spreadsheet = gc.open_by_key(SPREADSHEET_ID)
sheet = spreadsheet.worksheet(SHEET_NAME)

# Get the values in the last row
last_row_values = sheet.row_values(sheet.row_count)

# Update the last value in the last row (assuming it's a string, adjust accordingly)
new_value = 'NEW_VALUE'
last_row_values[-1] = new_value

# Update the last column of the last row in Google Sheets
sheet.update_row(sheet.row_count, last_row_values)


ValueError: ('Unexpected credentials type', None, 'Expected', 'service_account')